In [ ]:
#Imports
%matplotlib notebook

import sys
import math                

import numpy as np                  
from numpy.linalg import inv

from sklearn import linear_model as lm    
import pandas as pd
             
from statistics import mean

import statsmodels.regression.linear_model as sm

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_palette("colorblind")
color_blind_list = sns.color_palette("colorblind", 8)